In [ ]:
import cv2
from PIL import Image
import numpy as np
import os

# Load the original image
image_path = "/home/princeton/Downloads/captone/Final_project/Princeton.jpg"
image = cv2.imread(image_path)

# Define the directory to save augmented images
output_dir = "augmented_images"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define the number of augmented images you want to generate
num_augmented_images = 100

# Perform image augmentation
for i in range(num_augmented_images):
    # Apply random transformations to the image
    augmented_image = image.copy()
    
    # Example transformations (you can customize these as needed)
    # Rotate the image by a random angle between -10 and 10 degrees
    angle = np.random.randint(-10, 10)
    augmented_image = Image.fromarray(augmented_image)
    augmented_image = augmented_image.rotate(angle, resample=Image.BICUBIC, fillcolor=(255, 255, 255))
    augmented_image = np.array(augmented_image)
    
    # Save the augmented image
    output_path = os.path.join(output_dir, f"augmented_image_{i}.jpg")
    cv2.imwrite(output_path, augmented_image)

print("Image augmentation completed.")

In [19]:
# load dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [20]:
# load and create labeled data
data_dir = '/home/princeton/Downloads/captone/Final_project/Images'
batch_size = 32
epochs = 20
input_shape = (224, 224, 3)
image_paths = []
labels = []

for category in os.listdir(data_dir):
    category_dir = os.path.join(data_dir, category)
    if os.path.isdir(category_dir):
        for image_filename in os.listdir(category_dir):
            if image_filename.endswith('.jpg'):
                image_path = os.path.join(category_dir, image_filename)
                image_paths.append(image_path)
                labels.append(category)

In [21]:
# split dataset
train_image_paths, test_image_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42)

In [22]:
# create data generator for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_dataframe(
    pd.DataFrame({'image_path': train_image_paths, 'label': train_labels}),
    x_col='image_path',
    y_col='label',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

Found 480 validated image filenames belonging to 6 classes.


In [27]:
# build model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))  # 5 classes (4 diseases + 1 healthy)

In [28]:
from tensorflow.keras.optimizers import Adam

# compile 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0001),
              metrics=['accuracy'])

In [29]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the filepath with '.keras' extension
checkpoint = ModelCheckpoint('Face-Recognition-based-Attendance-System_model.h5.keras',
                             monitor='val_loss',
                             save_best_only=True,
                             verbose=1)

In [30]:
# train model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    callbacks=[checkpoint]
)

Epoch 1/20


/home/princeton/Downloads/captone/cap_env/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-04-10 22:34:58.710937: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:42: Filling up shuffle buffer (this may take a while): 5 of 8
2024-04-10 22:35:05.672217: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


15/15 ━━━━━━━━━━━━━━━━━━━━ 81s 4s/step - accuracy: 0.1160 - loss: 1.9028
Epoch 2/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/20


/home/princeton/Downloads/captone/cap_env/lib/python3.11/site-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
2024-04-10 22:36:07.208510: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/home/princeton/miniconda3/lib/python3.11/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-04-10 22:36:21.539443: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:42: Filling up shuffle buffer (this may take a while): 5 of 8
2024-04-10 22:36:28.786865: I tensorflow/core/kernels/data/shuffle_dat

15/15 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - accuracy: 0.2412 - loss: 1.7597
Epoch 4/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 504us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/20


...

2024-04-10 22:48:33.374792: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-10 22:48:44.779768: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:42: Filling up shuffle buffer (this may take a while): 5 of 8
2024-04-10 22:48:52.686784: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


15/15 ━━━━━━━━━━━━━━━━━━━━ 83s 4s/step - accuracy: 0.3159 - loss: 1.2567
Epoch 18/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 19/20


2024-04-10 22:49:56.388693: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-10 22:51:07.297144: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:42: Filling up shuffle buffer (this may take a while): 5 of 8
2024-04-10 22:51:14.093709: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


15/15 ━━━━━━━━━━━━━━━━━━━━ 139s 4s/step - accuracy: 0.3362 - loss: 1.2390
Epoch 20/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 817us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00


2024-04-10 22:52:15.029308: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [31]:
# save model
model.save('Face-Recognition-based-Attendance-System_model.h5')

In [32]:
# evaluate on test data
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_dataframe(
    pd.DataFrame({'image_path': test_image_paths, 'label': test_labels}),
    x_col='image_path',
    y_col='label',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

loss, accuracy = model.evaluate(test_generator)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy * 100:.2f}%')

Found 120 validated image filenames belonging to 6 classes.


/home/princeton/Downloads/captone/cap_env/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.3002 - loss: 1.1869
Test loss: 1.1915
Test accuracy: 30.00%


In [33]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

In [35]:
# Load the pre-trained CNN model
model = load_model("/home/princeton/Downloads/captone/Final_project/Face-Recognition-based-Attendance-System_model.h5")

In [36]:
# Compile the model (if not compiled before loading)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [37]:
# Load the image you want to classify
img_path = "/home/princeton/Downloads/captone/Final_project/Images/Preyas/augmented_image_0.jpg"  # Provide the path to your image
img = image.load_img(img_path, target_size=(224, 224))  # Assuming input shape of the model is (224, 224)

In [38]:
# Preprocess the image
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Normalize the pixel values

In [39]:
# Use the model to predict the class of the image
predictions = model.predict(img_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step


In [40]:
# Map the predicted class index to the corresponding category label
class_labels = ['Kunal', 'Preyas', 'Princeton', 'Sanjay', 'Vandana']
predicted_class_index = np.argmax(predictions)
predicted_class_label = class_labels[predicted_class_index]

print("Predicted class:", predicted_class_label)

Predicted class: Preyas
